# This notebook is an IDA of orders.csv

## 1. Dataset Overview

We start by examining the first few rows to get a sense of the data.

- 51,290 rows
- 24 columns, including:
  - Order details (Order ID, Order Date, Ship Date, Ship Mode, Order Priority)
  - Customer details (Customer ID, Customer Name, Segment)
  - Product details (Product ID, Product Name, Category, Sub-Category)
  - Financials (Sales, Profit, Discount, Shipping Cost)
  - Geographical info (City, State, Country, Postal Code, Region, Market)


In [7]:
import pandas as pd
import numpy as np

orders = pd.read_csv("../data/raw/orders.csv")

print(orders.head())

   Row ID                Order ID  Order Date   Ship Date       Ship Mode  \
0   24599  IN-2017-CA120551-42816  2017-03-22  2017-03-29  Standard Class   
1   29465  ID-2015-BD116051-42248  2015-09-01  2015-09-04    Second Class   
2   24598  IN-2017-CA120551-42816  2017-03-22  2017-03-29  Standard Class   
3   24597  IN-2017-CA120551-42816  2017-03-22  2017-03-29  Standard Class   
4   29464  ID-2015-BD116051-42248  2015-09-01  2015-09-04    Second Class   

  Customer ID    Customer Name      Segment  Postal Code   City  ...  \
0   CA-120551  Cathy Armstrong  Home Office          NaN  Herat  ...   
1   BD-116051     Brian Dahlen     Consumer          NaN  Herat  ...   
2   CA-120551  Cathy Armstrong  Home Office          NaN  Herat  ...   
3   CA-120551  Cathy Armstrong  Home Office          NaN  Herat  ...   
4   BD-116051     Brian Dahlen     Consumer          NaN  Herat  ...   

    Product ID                                       Product Name  \
0  FUR-BO-4861                    I

In [8]:
print(orders.columns)

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Postal Code', 'City',
       'State', 'Country', 'Region', 'Market', 'Product ID', 'Product Name',
       'Sub-Category', 'Category', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Shipping Cost', 'Order Priority'],
      dtype='str')


### Observation:

All expected columns are present, including **foreign key candidates** for customers (`Customer ID`) and products (`Product ID`) which will help in building a star schema.


In [9]:
print(orders.shape)

(51290, 24)


- Dataset contains **51,290 rows and 24 columns**.

In [10]:
print(orders.isnull().sum())

Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
Postal Code       41296
City                  0
State                 0
Country               0
Region                0
Market                0
Product ID            0
Product Name          0
Sub-Category          0
Category              0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
dtype: int64


### Observations:

- Most columns are fully populated.
- `Postal Code` has **41,296 missing values** (~80% of rows).  
  This column may need special handling or could be excluded from modeling.


In [11]:
print(orders.info())

<class 'pandas.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  str    
 2   Order Date      51290 non-null  str    
 3   Ship Date       51290 non-null  str    
 4   Ship Mode       51290 non-null  str    
 5   Customer ID     51290 non-null  str    
 6   Customer Name   51290 non-null  str    
 7   Segment         51290 non-null  str    
 8   Postal Code     9994 non-null   float64
 9   City            51290 non-null  str    
 10  State           51290 non-null  str    
 11  Country         51290 non-null  str    
 12  Region          51290 non-null  str    
 13  Market          51290 non-null  str    
 14  Product ID      51290 non-null  str    
 15  Product Name    51290 non-null  str    
 16  Sub-Category    51290 non-null  str    
 17  Category        51290 non-null  str    
 1

### Observation:

- 17 string columns, 5 floats, 2 integers.
- Dates (`Order Date` and `Ship Date`) are currently strings and should be converted to datetime format for analysis.


In [12]:
print(orders.describe())

            Row ID   Postal Code         Sales      Quantity      Discount  \
count  51290.00000   9994.000000  51290.000000  51290.000000  51290.000000   
mean   25645.50000  55190.379428    246.490581      3.476545      0.142908   
std    14806.29199  32063.693350    487.565361      2.278766      0.212280   
min        1.00000   1040.000000      0.444000      1.000000      0.000000   
25%    12823.25000  23223.000000     30.758625      2.000000      0.000000   
50%    25645.50000  56430.500000     85.053000      3.000000      0.000000   
75%    38467.75000  90008.000000    251.053200      5.000000      0.200000   
max    51290.00000  99301.000000  22638.480000     14.000000      0.850000   

             Profit  Shipping Cost  
count  51290.000000   51290.000000  
mean      28.610982      26.478567  
std      174.340972      57.251373  
min    -6599.978000       1.002000  
25%        0.000000       2.610000  
50%        9.240000       7.790000  
75%       36.810000      24.450000  
m

### Observation:

- `Sales`: Mean ≈ 246.5, Max = 22,638  
- `Profit`: Mean ≈ 28.6, but min = -6,599 → large negative profits, may indicate returns or discounts  
- `Quantity`: Mostly low (1–14 units per order)  
- `Discount`: Mostly 0, max 85%


In [13]:
print(orders['Order ID'].nunique())

25728


### Observation:

- 25,728 unique orders  
- Each order may contain multiple line items, explaining why rows > unique orders


## 2. Next Steps / Recommendations

1. **Data Cleaning**
   - Convert `Order Date` and `Ship Date` to datetime
   - Handle `Postal Code` missing values
   - Check for duplicate rows (especially since some `Order ID`s repeat)

2. **Data Validation**
   - Ensure all `Customer ID`s exist in `customers.csv`
   - Ensure all `Product ID`s exist in `products.csv` (or `returns.csv` if relevant)

3. **Feature Engineering Prep**
   - Compute `Order Duration` = `Ship Date - Order Date`
   - Categorize `Profit`/`Sales` into bins if needed
   - Extract month, quarter, weekday from `Order Date`

This notebook sets the foundation for **ETL transformations** and **analytics modeling** in later layers of the GlobalRetail 360 project.
